# Embeddings

Em uma postagem anterior sobre [tokens](https://maximofn.com/tokens/), já vimos a representação mínima de cada palavra. O que corresponde a atribuir um número à divisão mínima de cada palavra.

Entretanto, os transformadores e, portanto, os LLMs, não representam as informações das palavras dessa forma, mas o fazem por meio de "embeddings".

Este caderno foi traduzido automaticamente para torná-lo acessível a mais pessoas, por favor me avise se você vir algum erro de digitação..

Primeiramente, examinaremos duas maneiras de representar palavras dentro de transformadores: `codificação ordinal` e `codificação de um hot`. E, analisando os problemas desses dois tipos de representações, poderemos chegar às "incorporações".

## Codificação ordinal

Essa é a maneira mais básica de representar as palavras dentro dos transformadores. Ela consiste em atribuir um número a cada palavra ou manter os números já atribuídos aos tokens.

No entanto, esse tipo de representação tem dois problemas

 * Imaginemos que table corresponda ao token 3, cat ao token 1 e dog ao token 2. Poderíamos supor que `table = cat + dog`, mas esse não é o caso. Não existe essa relação entre essas palavras. Poderíamos até pensar que, atribuindo os tokens corretos, essa relação poderia ocorrer. No entanto, essa ideia não funciona com palavras que têm mais de um significado, como a palavra "banco".

 * O segundo problema é que as redes neurais fazem internamente muitos cálculos numéricos, portanto, pode ser que mesa tenha o token 3 e seja internamente mais importante do que a palavra cat que tem o token 1.

Portanto, esse tipo de representação de palavras pode ser descartado muito rapidamente.

## Uma codificação quente

O que você faz aqui é usar vetores de dimensão `N`. Por exemplo, vimos que o OpenAI tem um vocabulário de `100277` tokens distintos. Portanto, se usarmos "one hot encoding", cada palavra será representada por um vetor de "100277" dimensões.

No entanto, uma codificação quente tem dois outros problemas importantes.

 * Ele não leva em conta a relação entre as palavras. Portanto, se tivermos duas palavras que sejam sinônimas, por exemplo, `cat` e `feline`, teremos dois vetores diferentes para representá-las.
 No idioma, a relação entre as palavras é muito importante, e não levar essa relação em conta é um grande problema.

 * O segundo problema é que os vetores são muito grandes. Se tivermos um vocabulário de `100277` tokens, cada palavra será representada por um vetor de `100277` dimensões. Isso torna os vetores muito grandes e os cálculos muito caros. Além disso, esses vetores serão todos zeros, exceto na posição correspondente ao token da palavra. Portanto, a maioria dos cálculos serão multiplicações por zero, que são cálculos que não somam nada. Portanto, teremos muita memória alocada para vetores em que só há um 1 em uma determinada posição.

## Embeddings de palavras

Os word embeddings tentam resolver os problemas dos dois tipos anteriores de representações. Para isso, são usados vetores de `N` dimensões, mas, nesse caso, não são usados vetores de 100277 dimensões, mas sim vetores de muito menos dimensões. Por exemplo, veremos que o OpenAI usa `1536` dimensões.

Cada uma das dimensões desses vetores representa uma característica da palavra. Por exemplo, uma dimensão pode representar se a palavra é um verbo ou um substantivo. Outra dimensão pode representar se a palavra é um animal ou não. Outra dimensão pode representar se a palavra é um substantivo próprio ou não. E assim por diante.

No entanto, esses recursos não são definidos manualmente, mas aprendidos automaticamente. Durante o treinamento dos transformadores, os valores de cada uma das dimensões dos vetores são ajustados, de modo que as características de cada uma das palavras sejam aprendidas.

Ao fazer com que cada uma das dimensões da palavra represente uma característica da palavra, as palavras que têm características semelhantes terão vetores semelhantes. Por exemplo, as palavras `cat` e `feline` terão vetores muito semelhantes, pois ambas são animais. E as palavras `table` e `chair` terão vetores semelhantes, pois ambas são móveis.

Na imagem a seguir, podemos ver uma representação tridimensional das palavras e podemos ver que todas as palavras relacionadas a `school` estão próximas, todas as palavras relacionadas a `food` estão próximas e todas as palavras relacionadas a `ball` estão próximas.

![word_embedding_3_dimmension](http://maximofn.com/wp-content/uploads/2023/12/word_embedding_3_dimmension.webp)

Como cada uma das dimensões dos vetores representa uma característica da palavra, podemos realizar operações com palavras. Por exemplo, se subtrairmos a palavra "king" (rei) da palavra "man" (homem) e adicionarmos a palavra "woman" (mulher), obteremos uma palavra muito semelhante à palavra "queen" (rainha). Verificaremos isso mais tarde com um exemplo

### Similaridade entre palavras

Como cada palavra é representada por um vetor N-dimensional, podemos calcular a semelhança entre duas palavras. Para isso, é usada a função de similaridade de cosseno.

Se duas palavras estiverem próximas no espaço vetorial, isso significa que o ângulo entre seus vetores é pequeno, portanto, seu cosseno é próximo de 1. Se houver um ângulo de 90 graus entre os vetores, o cosseno será 0, o que significa que não há semelhança entre as palavras. E se houver um ângulo de 180 graus entre os vetores, o cosseno será -1, ou seja, as palavras são opostas.

![cosine similarity](http://maximofn.com/wp-content/uploads/2023/12/cosine_similarity-scaled.webp)

### Exemplo com embeddings da OpenAI

Agora que já sabemos o que são `embeddings`, vamos dar uma olhada em alguns exemplos com os `embeddings` fornecidos pela `API` `OpenAI`.

Para fazer isso, primeiro precisamos ter o pacote `OpenAI` instalado.

````bash
pip install openai
```

Importamos as bibliotecas necessárias

In [1]:
from openai import OpenAI
import torch
from torch.nn.functional import cosine_similarity

Usamos uma "chave API" da OpenAI. Para fazer isso, vá para a página [OpenAI] (https://openai.com/) e registre-se. Depois de registrado, vá para a seção [API Keys](https://platform.openai.com/api-keys) e crie uma nova `API Key`.

![open ai api key](https://raw.githubusercontent.com/maximofn/alfred/main/gifs/openaix2.gif)

In [2]:
api_key = "Pon aquí tu API key"

Selecionamos o modelo de embeddings que queremos usar. Nesse caso, usaremos o `text-embedding-ada-002`, que é recomendado pela `OpenAI` em sua documentação [embeddings] (https://platform.openai.com/docs/guides/embeddings/).

In [ ]:
model_openai = "text-embedding-ada-002"

Criar um cliente `API

In [ ]:
client_openai = OpenAI(api_key=api_key, organization=None)

Vamos ver como são os `embeddings` da palavra `King`.

In [7]:
word = "Rey"
embedding_openai = torch.Tensor(client_openai.embeddings.create(input=word, model=model_openai).data[0].embedding)

embedding_openai.shape, embedding_openai

(torch.Size([1536]),
 tensor([-0.0103, -0.0005, -0.0189,  ..., -0.0009, -0.0226,  0.0045]))

Como podemos ver, obtemos um vetor de `1536` dimensões.

### Operações com palavras

Vamos obter os embeddings das palavras `king`, `man`, `woman` e `Queen`.

In [19]:
embedding_openai_rey = torch.Tensor(client_openai.embeddings.create(input="rey", model=model_openai).data[0].embedding)
embedding_openai_hombre = torch.Tensor(client_openai.embeddings.create(input="hombre", model=model_openai).data[0].embedding)
embedding_openai_mujer = torch.Tensor(client_openai.embeddings.create(input="mujer", model=model_openai).data[0].embedding)
embedding_openai_reina = torch.Tensor(client_openai.embeddings.create(input="reina", model=model_openai).data[0].embedding)

In [20]:
embedding_openai_reina.shape, embedding_openai_reina

(torch.Size([1536]),
 tensor([-0.0110, -0.0084, -0.0115,  ...,  0.0082, -0.0096, -0.0024]))

Vamos obter a incorporação resultante da subtração da incorporação de "homem" de "rei" e da adição da incorporação de "mulher" a "rei".

In [21]:
embedding_openai = embedding_openai_rey - embedding_openai_hombre + embedding_openai_mujer

In [22]:
embedding_openai.shape, embedding_openai

(torch.Size([1536]),
 tensor([-0.0226, -0.0323,  0.0017,  ...,  0.0014, -0.0290, -0.0188]))

Por fim, comparamos o resultado obtido com a incorporação da `reina`. Para isso, usamos a função `cosine_similarity` fornecida pela biblioteca `pytorch`.

In [23]:
similarity_openai = cosine_similarity(embedding_openai.unsqueeze(0), embedding_openai_reina.unsqueeze(0)).item()

print(f"similarity_openai: {similarity_openai}")

similarity_openai: 0.7564167976379395


Como podemos ver, é um valor muito próximo de 1, portanto, podemos dizer que o resultado obtido é muito semelhante à incorporação da `reina`.

Se usarmos palavras em inglês, obteremos um resultado mais próximo de 1.

In [15]:
embedding_openai_rey = torch.Tensor(client_openai.embeddings.create(input="king", model=model_openai).data[0].embedding)
embedding_openai_hombre = torch.Tensor(client_openai.embeddings.create(input="man", model=model_openai).data[0].embedding)
embedding_openai_mujer = torch.Tensor(client_openai.embeddings.create(input="woman", model=model_openai).data[0].embedding)
embedding_openai_reina = torch.Tensor(client_openai.embeddings.create(input="queen", model=model_openai).data[0].embedding)

In [16]:
embedding_openai = embedding_openai_rey - embedding_openai_hombre + embedding_openai_mujer

In [17]:
similarity_openai = cosine_similarity(embedding_openai.unsqueeze(0), embedding_openai_reina.unsqueeze(0))
print(f"similarity_openai: {similarity_openai}")

similarity_openai: tensor([0.8849])


Isso é normal, pois o modelo OpenAi foi treinado com mais txt em inglês do que em espanhol.

### Tipos de incorporação de palavras

Há vários tipos de word embeddings, e cada um deles tem suas vantagens e desvantagens. Vamos dar uma olhada nos mais importantes

 * Word2Vec
 * GloVe
 * FastText
 * BERT
 * GPT-2

#### Word2Vec

O Word2Vec é um algoritmo usado para criar embeddings de palavras. Esse algoritmo foi criado pelo Google em 2013 e é um dos algoritmos mais usados para criar word embeddings.

Ele tem duas variantes, `CBOW` e `Skip-gram`. O `CBOW` é mais rápido de treinar, enquanto o `Skip-gram` é mais preciso. Vamos dar uma olhada em como cada um funciona

##### CBOW

CBOW ou `Continuous Bag of Words` é um algoritmo usado para prever uma palavra a partir das palavras ao redor. Por exemplo, se tivermos a frase `O gato é um animal`, o algoritmo tentará prever a palavra `gato` a partir das palavras ao redor, nesse caso `O`, `é`, `um` e `animal`.

![CBOW](http://maximofn.com/wp-content/uploads/2023/12/cbow-scaled.webp)

Nessa arquitetura, o modelo prevê qual é a palavra mais provável em um determinado contexto. Portanto, as palavras que têm a mesma probabilidade de ocorrência são consideradas semelhantes e, portanto, estão mais próximas no espaço dimensional.

Suponha que, em uma frase, substituamos `boat` por `boat`, então o modelo prevê a probabilidade de ambos e, se for semelhante, podemos considerar que as palavras são semelhantes.

##### Skip-gram

O `Skip-gram` ou `Skip-gram with Negative Sampling` é um algoritmo usado para prever as palavras ao redor de uma palavra. Por exemplo, se tivermos a frase `O gato é um animal`, o algoritmo tentará prever as palavras `O`, `é`, `um` e `animal` a partir da palavra `gato`.

Skip-gram](http://maximofn.com/wp-content/uploads/2023/12/Skip-gram-scaled.webp)

Essa arquitetura é semelhante à do CBOW, mas o modelo funciona de forma inversa. O modelo prevê o contexto usando a palavra dada. Portanto, as palavras que têm o mesmo contexto são consideradas semelhantes e, portanto, estão mais próximas no espaço dimensional.

#### GloVe

GloVe` ou `Global Vectors for Word Representation` é um algoritmo usado para criar embeddings de palavras. Esse algoritmo foi criado pela Universidade de Stanford em 2014.

O Word2Vec ignora o fato de que algumas palavras de contexto ocorrem com mais frequência do que outras e também leva em conta apenas o contexto local e, portanto, não captura o contexto global.

Esse algoritmo usa uma matriz de co-ocorrência para criar os embeddings de palavras. Essa matriz de co-ocorrência é uma matriz que contém o número de vezes que cada palavra aparece ao lado de cada uma das outras palavras do vocabulário.

#### FastText

O FastText é um algoritmo usado para criar incorporação de palavras. Esse algoritmo foi criado pelo Facebook em 2016.

Uma das principais desvantagens do `Word2Vec` e do `GloVe` é que eles não podem codificar palavras desconhecidas ou fora do vocabulário.

Portanto, para lidar com esse problema, o Facebook propôs um modelo `FastText`. Ele é uma extensão do `Word2Vec` e segue o mesmo modelo `Skip-gram` e `CBOW`, mas, ao contrário do `Word2Vec`, que alimenta a rede neural com palavras inteiras, o `FastText` primeiro divide as palavras em várias subpalavras (ou `n-gramas`) e depois as alimenta na rede neural.

Por exemplo, se o valor de `n` for 3 e a palavra for `apple`, então seu tri-grama será [`<ma`, `man`, `anz`, `nza`, `zan`, `ana`, `na>`] e sua incorporação de palavras será a soma da representação vetorial desses tri-gramas. Aqui, os hiperparâmetros `min_n` e `max_n` são considerados como 3 e os caracteres `<` e `>` representam o início e o fim da palavra.

Portanto, usando essa metodologia, as palavras desconhecidas podem ser representadas em forma de vetor, pois há uma alta probabilidade de que seus `n-gramas` também estejam presentes em outras palavras.

Esse algoritmo é um aprimoramento do `Word2Vec`, pois, além de levar em conta as palavras ao redor de uma palavra, ele também leva em conta os `n-gramas` da palavra. Por exemplo, se tivermos a palavra `cat`, ele também leva em conta os `n-gramas` da palavra, nesse caso `ga`, `at` e `to`, para `n = 2`.

#### Limitações da incorporação de palavras

As técnicas de incorporação de palavras produziram um resultado decente, mas o problema é que a abordagem não é suficientemente precisa. Elas não levam em conta a ordem em que as palavras aparecem, o que leva à perda da compreensão sintática e semântica da frase.

Por exemplo, `Você vai lá para ensinar, não para brincar` E `Você vai lá para brincar, não para ensinar` Ambas as frases terão a mesma representação no espaço vetorial, mas não significam a mesma coisa.

Além disso, o modelo de incorporação de palavras não pode fornecer resultados satisfatórios em uma grande quantidade de dados de texto, pois a mesma palavra pode ter um significado diferente em uma frase diferente, dependendo do contexto da frase.

Por exemplo, `I am going to sit in the bank` E `I am going to do business in the bank` Em ambas as frases, a palavra `bank` tem significados diferentes.

Portanto, precisamos de um tipo de representação que possa reter o significado contextual da palavra presente em uma frase.

## Embeddings de frases

A incorporação de frases é semelhante à incorporação de palavras, mas, em vez de palavras, ela codifica a frase inteira na representação vetorial.

Uma maneira simples de obter a incorporação de frases é calcular a média das incorporações de palavras de todas as palavras presentes na frase. Mas isso não é suficientemente preciso.

Alguns dos modelos mais avançados de incorporação de frases são o `ELMo`, o `InferSent` e o `Sentence-BERT`.

### ELMo

ELMo` ou `Embeddings from Language Models` é um modelo de incorporação de frases criado pela Allen University em 2018. Ele usa uma rede LSTM profunda bidirecional para produzir representação vetorial. O `ELMo` pode representar palavras desconhecidas ou fora do vocabulário em forma de vetor, pois é baseado em caracteres.

### InferSent

O InferSent é um modelo de incorporação de frases criado pelo Facebook em 2017. Ele usa uma rede LSTM profunda bidirecional para produzir representação vetorial. O `InferSent` pode representar palavras desconhecidas ou fora do vocabulário em forma de vetor, pois é baseado em caracteres. As frases são codificadas em uma representação vetorial de 4096 dimensões.

O treinamento do modelo é feito com o conjunto de dados Stanford Natural Language Inference (`SNLI`). Esse conjunto de dados é rotulado e escrito por humanos para cerca de 500 mil pares de frases.

### Sentença-BERT

O Sentence-BERT é um modelo de incorporação de frases criado pela Universidade de Londres em 2019. Ele usa uma rede LSTM profunda bidirecional para produzir representação vetorial. O `Sentence-BERT` pode representar palavras desconhecidas ou fora do vocabulário na forma vetorial, pois é baseado em caracteres. As frases são codificadas em uma representação vetorial de 768 dimensões.

O modelo de PNL de última geração `BERT` é excelente em tarefas de Semantic Textual Similarity, mas o problema é que ele levaria muito tempo para um corpus enorme (65 horas para 10.000 frases), pois exige que ambas as frases sejam inseridas na rede, o que aumenta o cálculo em um fator enorme.

Portanto, o `Sentence-BERT` é uma modificação do modelo `BERT`.